In [ ]:
def T(formula):
    def negate_expression(expression):
        return '¬' + expression

    def process_binary_operation(expression, operator, operation_func):
        parts = expression.split(operator)
        left_expr = transform(parts[0].strip())
        right_expr = transform(parts[1].strip())
        return operation_func(left_expr, right_expr)

    def apply_conjunction(left_expr, right_expr):
        return left_expr + ' ∧ ' + right_expr

    def apply_disjunction(left_expr, right_expr):
        return '¬(¬' + left_expr + ' ∧ ¬' + right_expr + ')'

    def apply_implication(left_expr, right_expr):
        return '¬(' + left_expr + ' ∧ ¬' + right_expr + ')'

    # Recursive transformation
    def transform(expression):
        # Base case: single proposition
        if expression.isalpha():
            return expression

        # Remove outer parentheses
        while expression.startswith('(') and expression.endswith(')'):
            expression = expression[1:-1]

        # Recursive transformation for negation
        if expression.startswith('¬'):
            inner_expr = transform(expression[1:])
            return negate_expression(inner_expr)

        # Recursive transformation for binary operations
        for op, operation_func in [('∧', apply_conjunction), ('∨', apply_disjunction), ('→', apply_implication)]:
            if op in expression:
                return process_binary_operation(expression, op, operation_func)

        return expression

    # Start the transformation process
    return transform(formula)

In [ ]:
class ExpressionNode:
    def __init__(self, content, left_child=None, right_child=None):
        self.content = content
        self.left_child = left_child
        self.right_child = right_child

def parse_logical_expression(expression):
    def strip_outer_parentheses(expression):
        while expression.startswith('(') and expression.endswith(')'):
            expression = expression[1:-1].strip()
        return expression

    def locate_main_conjunction(expression):
        depth = 0
        for i, char in enumerate(expression):
            if char == '(':
                depth += 1
            elif char == ')':
                depth -= 1
            elif depth == 0 and char == '∧':
                return i
        return -1

    expression = strip_outer_parentheses(expression)
    conjunction_index = locate_main_conjunction(expression)

    if conjunction_index != -1:
        return ExpressionNode('∧',
                              parse_logical_expression(expression[:conjunction_index].strip()),
                              parse_logical_expression(expression[conjunction_index+1:].strip()))
    elif expression.startswith('¬'):
        return ExpressionNode('¬', right_child=parse_logical_expression(expression[1:].strip()))
    else:
        return ExpressionNode(expression)

def transform_to_dag(expression_node, memoized_subtrees=None):
    if expression_node is None:
        return None

    if memoized_subtrees is None:
        memoized_subtrees = {}

    left_subtree = transform_to_dag(expression_node.left_child, memoized_subtrees)
    right_subtree = transform_to_dag(expression_node.right_child, memoized_subtrees)
    node_signature = (expression_node.content, left_subtree, right_subtree)

    if node_signature in memoized_subtrees:
        return memoized_subtrees[node_signature]

    expression_node.left_child = left_subtree
    expression_node.right_child = right_subtree
    memoized_subtrees[node_signature] = expression_node

    return expression_node

In [ ]:
def force_laws(node, truth_values, processed_nodes):
    if node is None or node in processed_nodes:
        return True

    processed_nodes.add(node)

    if node.content == '¬':
        negated_node = node.right_child  # Assuming negation affects only the right child
        if negated_node not in truth_values:  # If the negated node hasn't been evaluated
            truth_values[negated_node] = not truth_values.get(node, True)  # Assign the opposite truth value
        elif truth_values[negated_node] == truth_values.get(node, True):  # Check for logical inconsistency
            return False  # Inconsistency found

    elif node.content == '∧':
        left_child, right_child = node.left_child, node.right_child
        if truth_values.get(node, True):  # If conjunction node is true, both children must be true
            truth_values[left_child] = truth_values[right_child] = True
        else:
            # If conjunction node is false, at least one child must be false
            if left_child in truth_values and right_child in truth_values and truth_values[left_child] and truth_values[right_child]:
                return False  # Inconsistency found

    # Recursively apply truth values to children nodes
    return force_laws(node.left_child, truth_values, processed_nodes) and force_laws(node.right_child, truth_values, processed_nodes)

def check_satisfiability(expression_root):
    truth_values = {expression_root: True}  # Initialize with the root node assumed to be true
    processed_nodes = set()

    # Propagate truth values through the expression tree and check for inconsistencies
    if not force_laws(expression_root, truth_values, processed_nodes):
        return False  # Inconsistency found, formula is not satisfiable

    return True  # No inconsistencies found, formula is satisfiable

In [ ]:
def print_tree(node, prefix="", branch='root'):
    if node is not None:
        branch_symbols = {"root": "", "left": "├── ", "right": "└── "}
        connectors = {"root": "    ", "left": "│   ", "right": "    "}

        print(prefix + branch_symbols[branch] + node.content)

        prefix += connectors[branch]

        if node.left_child and node.right_child:
            print_tree(node.left_child, prefix, 'left')
            print_tree(node.right_child, prefix, 'right')
        elif node.left_child:
            print_tree(node.left_child, prefix, 'right')
        elif node.right_child:
            print_tree(node.right_child, prefix, 'right')

def print_dag(node, prefix="", branch='root', visited_nodes=None, node_labels=None):
    if node is None:
        return

    if visited_nodes is None:
        visited_nodes = set()
        node_labels = {}

    if node not in node_labels:
        node_labels[node] = len(node_labels) + 1

    node_label = node_labels[node]

    branch_symbols = {"root": "", "left": "├── ", "right": "└── "}
    connectors = {"root": "    ", "left": "│   ", "right": "    "}

    is_shared = ' (shared)' if node in visited_nodes else ''
    print(prefix + branch_symbols[branch] + f"Node {node_label}: {node.content}{is_shared}")

    if node in visited_nodes:
        return

    visited_nodes.add(node)

    prefix += connectors[branch]

    if node.left_child and node.right_child:
        print_dag(node.left_child, prefix, 'left', visited_nodes, node_labels)
        print_dag(node.right_child, prefix, 'right', visited_nodes, node_labels)
    elif node.left_child:
        print_dag(node.left_child, prefix, 'right', visited_nodes, node_labels)
    elif node.right_child:
        print_dag(node.right_child, prefix, 'right', visited_nodes, node_labels)

# ***Example I***

In [ ]:
input_formula = "p ∧ ¬(q → ¬p)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∧ ¬(q → ¬p)
Translated formula: p ∧ ¬¬(q ∧ ¬¬p)


In [ ]:
input_formula = translated_formula
root = parse_logical_expression(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
∧
    ├── p
    └── ¬
        └── ¬
            └── ∧
                ├── q
                └── ¬
                    └── ¬
                        └── p


In [ ]:
seen_subtrees = {}
root = transform_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
    ├── Node 2: p
    └── Node 3: ¬
        └── Node 4: ¬
            └── Node 5: ∧
                ├── Node 6: q
                └── Node 7: ¬
                    └── Node 8: ¬
                        └── Node 2: p (shared)


In [ ]:
satisfiable = check_satisfiability(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


# ***Example 2***

In [ ]:
input_formula = "p ∧ ¬(q ∨ ¬p)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∧ ¬(q ∨ ¬p)
Translated formula: p ∧ ¬¬(¬q ∧ ¬¬p)


In [ ]:
input_formula = translated_formula
root = parse_logical_expression(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
∧
    ├── p
    └── ¬
        └── ¬
            └── ∧
                ├── ¬
                │   └── q
                └── ¬
                    └── ¬
                        └── p


In [ ]:
seen_subtrees = {}
root = transform_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
    ├── Node 2: p
    └── Node 3: ¬
        └── Node 4: ¬
            └── Node 5: ∧
                ├── Node 6: ¬
                │   └── Node 7: q
                └── Node 8: ¬
                    └── Node 9: ¬
                        └── Node 2: p (shared)


In [ ]:
satisfiable = check_satisfiability(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


# ***Example 3***

In [ ]:
input_formula = "p ∧ ¬p"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∧ ¬p
Translated formula: p ∧ ¬p


In [ ]:
input_formula = translated_formula
root = parse_logical_expression(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
∧
    ├── p
    └── ¬
        └── p


In [ ]:
seen_subtrees = {}
root = transform_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
    ├── Node 2: p
    └── Node 3: ¬
        └── Node 2: p (shared)


In [ ]:
satisfiable = check_satisfiability(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is not satisfiable.


# ***Example 4***

In [ ]:
input_formula = "¬p ∧ (q ∨ r)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: ¬p ∧ (q ∨ r)
Translated formula: ¬p ∧ ¬(¬q ∧ ¬r)


In [ ]:
input_formula = translated_formula
root = parse_logical_expression(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
∧
    ├── ¬
    │   └── p
    └── ¬
        └── ∧
            ├── ¬
            │   └── q
            └── ¬
                └── r


In [ ]:
seen_subtrees = {}
root = transform_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
    ├── Node 2: ¬
    │   └── Node 3: p
    └── Node 4: ¬
        └── Node 5: ∧
            ├── Node 6: ¬
            │   └── Node 7: q
            └── Node 8: ¬
                └── Node 9: r


In [ ]:
satisfiable = check_satisfiability(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.
